In [ ]:
from tqdm import tqdm
from itertools import combinations
from pathlib import Path
import os
import sys
import warnings
import logging
from pprint import pformat
import traceback

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import MultipleLocator
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import silhouette_score, make_scorer
from scipy.stats import uniform
from sklearn.metrics import fbeta_score
import dynamic_yaml
import yaml

sys.path.append("/workspace/correlation-change-predict/ywt_library")
from data_module import gen_corr_dist_mat


current_dir = Path(os.getcwd())
data_config_path = current_dir / "../config/data_config.yaml"
with open(data_config_path) as f:
    data_cfg_yaml = dynamic_yaml.load(f)
    data_cfg = yaml.full_load(dynamic_yaml.dump(data_cfg_yaml))

warnings.simplefilter("ignore")
logging.basicConfig(level=logging.INFO)
matplotlib_logger = logging.getLogger("matplotlib")
matplotlib_logger.setLevel(logging.ERROR)
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False
# logger_list = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
# print(logger_list)

# %load_ext pycodestyle_magic
# %pycodestyle_on --ignore E501
logging.debug(pformat(data_cfg, indent=1, width=100, compact=True))

# Prepare data

## Data implement & output setting & testset setting

In [ ]:
res_dir = Path('./results/')
res_dir.mkdir(parents=True, exist_ok=True)


# setting of output files
# save_corr_data = True
# data implement setting
# data_implement = "SP500_20082017"  # ['BITCOIN_NVDA', 'PAPER_EVA_1', 'PAPER_EVA_2', 'PAPER_EVA_3',
                                                          # 'PAPER_EVA_4', 'PAPER_EVA_5', 'SP500_19972007', 'SP500_20082017',
                                                          # 'SP500_20082017_CONSUMER_DISCRETIONARY', 'TEST_CASE', TETUAN_POWER', 'TW50_20082018']
# etl set setting
# etl_items_setting = "-etl_all"  # -etl_train|-etl_all
# data split period setting, only suit for only settings of Korean paper
# data_split_setting = "-data_sp_test2"
# Decide how to calculate corr_ser
corr_ser_clac_method = "corr_ser_calc_regular" # corr_ser_calc_regular|corr_ser_calc_abs
# Decide composition of correlation_matrix
# corr_mat_compo = "sim"

#python data_module.py --data_implement SP500_20082017 --train_items_setting -train_all --data_split_setting -data_sp_test2 --graph_mat_compo sim --save_corr_data

In [ ]:
dataset_df = pd.read_csv(data_cfg["DATASETS"][data_implement]['FILE_PATH'])
dataset_df = dataset_df.set_index('Date')
all_set = list(dataset_df.columns)  # all data
train_set = data_cfg["DATASETS"][data_implement]['TRAIN_SET']
test_set = data_cfg['DATASETS'][data_implement]['TEST_SET'] if data_cfg['DATASETS'][data_implement].get('TEST_SET') else [p for p in all_set if p not in train_set]  # all data - train data
logging.info(f"===== len(train_set): {len(train_set)}, len(all_set): {len(all_set)}, len(test_set): {len(test_set)} =====")

# test items implement settings
items_implement = train_set if etl_items_setting == "-etl_train" else all_set
logging.info(f"===== len(etl set): {len(items_implement)} =====")

# setting of name of output files and pictures title
output_file_name = data_cfg["DATASETS"][data_implement]['OUTPUT_FILE_NAME_BASIS'] + etl_items_setting
fig_title = data_implement + etl_items_setting + data_split_setting
logging.info(f"===== file_name basis:{output_file_name}, fig_title basis:{fig_title} =====")
# display(dataset_df)

# output folder settings
corr_data_dir = Path(data_cfg["DIRS"]["PIPELINE_DATA_DIR"])/f"{output_file_name}-corr_data"
corr_data_dir.mkdir(parents=True, exist_ok=True)

## Load or Create Correlation Data

In [ ]:
data_length = int(len(dataset_df)/data_gen_cfg["CORR_WINDOW"])*data_gen_cfg["CORR_WINDOW"]
corr_ser_len_max = int((data_length-data_gen_cfg["CORR_WINDOW"])/data_gen_cfg["CORR_STRIDE"])
max_data_div_start_add = 0  # In the Korea paper, each pair has 5 corr_series(due to diversifing train data).
                            # BUT we only need to take one, so take 0 as arg.
corr_ind = []

# DEFAULT SETTING: data_gen_cfg["DATA_DIV_STRIDE"] == 20, data_gen_cfg["CORR_WINDOW"]==100, data_gen_cfg["CORR_STRIDE"]==100
data_end_init = corr_ser_len_max * data_gen_cfg["CORR_STRIDE"]
for i in range(0, max_data_div_start_add+1, data_gen_cfg["DATA_DIV_STRIDE"]):
    corr_ind.extend(list(range(data_gen_cfg["CORR_WINDOW"]-1+i, data_end_init+bool(i)*data_gen_cfg["CORR_STRIDE"], data_gen_cfg["CORR_STRIDE"])))  # only suit for settings of paper

train_df_path = corr_data_dir/f"{output_file_name}-corr_train.csv"
dev_df_path = corr_data_dir/f"{output_file_name}-corr_dev.csv"
test1_df_path = corr_data_dir/f"{output_file_name}-corr_test1.csv"
test2_df_path = corr_data_dir/f"{output_file_name}-corr_test2.csv"
all_corr_df_paths = dict(zip(["train_df", "dev_df", "test1_df", "test2_df"],
                             [train_df_path, dev_df_path, test1_df_path, test2_df_path]))
if all([df_path.exists() for df_path in all_corr_df_paths.values()]):
    corr_datasets = [pd.read_csv(df_path, index_col=["items"]) for df_path in all_corr_df_paths.values()]
else:
    corr_datasets = data_generation.gen_train_data(items_implement, raw_data_df=dataset_df, corr_ser_len_max=corr_ser_len_max, corr_df_paths=all_corr_df_paths, corr_ind=corr_ind, max_data_div_start_add=max_data_div_start_add, save_file=save_corr_data)

if data_split_setting == "-data_sp_test2":
    corr_dataset = corr_datasets[3]
    display(corr_dataset.head())

# Calculate properties of Corrlelation series

In [ ]:
# stl_decompn(corr_datasets[0].iloc[0,::], overview=True)
if corr_ser_clac_method == "corr_ser_calc_regular":
    corr_property_df_path = res_dir/f"{output_file_name}{data_split_setting}-corr_series_property.csv"
    corr_property_df = calc_corr_ser_property(corr_dataset=corr_dataset, corr_property_df_path=corr_property_df_path)
elif corr_ser_clac_method == "corr_ser_calc_abs":
    # calculate corr_property_df with abs(corr_dataset)
    corr_property_df_path = res_dir/f"{output_file_name}{data_split_setting}-corr_series_abs_property.csv"
    corr_property_df = calc_corr_ser_property(corr_dataset=corr_dataset.abs(), corr_property_df_path=corr_property_df_path)

logging.info(f"Min of corr_ser_mean:{corr_property_df.loc[::,'corr_ser_mean'].min()}")
display(corr_property_df.head())

# Clustring

## calculate dissimilarity matrix

In [ ]:
corr_mean = corr_property_df.loc[::,"corr_ser_mean"]
distance_mat = gen_corr_dist_mat(corr_mean, dataset_df, out_mat_compo=corr_mat_compo)
# test
# test_stock_tickers = ["ED", "BAC", "XEL", "MA"]
# test_distance_mat = distance_mat.loc[test_stock_tickers, test_stock_tickers]
# display(test_distance_mat)  # comlpete: (ED, BAC), (XEL), (MA) -> (ED, BAC), (XEL, MA)  -> (ED, BAC, XEL, MA)
#                             # single: (ED, BAC), (XEL), (MA) -> (ED, BAC, XEL), (MA)  -> (ED, BAC, XEL, MA)
logging.info(f"Min of distance_mat:{distance_mat.min()}")
display(distance_mat.head())

## calculate cluster label for each data

In [ ]:
def calc_silhouette_label_freq_std(estimator: "sklearn.cluster.<cluster_model>", X: "pd.DataFrame", silhouette_score_ratio: int = 0.1, silhouette_metric: str = "precomputed") -> float:
    estimator.fit(X)
    cluster_labels = estimator.labels_
    num_labels = len(set(cluster_labels))
    num_samples = len(X.index)
    labels_symbol, label_freq = np.unique(cluster_labels, return_counts=True)
    if num_labels == 1 or num_labels == num_samples:
        return -1
    else:
        return silhouette_score_ratio * silhouette_score(X, cluster_labels, metric=silhouette_metric) + (1 - silhouette_score_ratio) * (1 / np.array(label_freq).std())


def hrchy_clustering_distance_threshold_rs(X: "pd.DataFrame", data_mat_mode: str = "precomputed", verbose: int = 0):

    param_dict = {"n_clusters": [None], "affinity": [data_mat_mode],
                  "linkage": ["single", "complete", "average"],
                  "distance_threshold": uniform(loc=0.55, scale=0.6)}
    cv = [(slice(None), slice(None))]
    hrchy_clustering_rs = RandomizedSearchCV(estimator=AgglomerativeClustering(), param_distributions=param_dict,
                                             n_iter=100000, scoring=calc_silhouette_label_freq_std, cv=cv, n_jobs=-1)
    hrchy_clustering_rs.fit(X)

    if verbose==1:
        print(f"hrchy_clustering_rs.best_estimator_: {hrchy_clustering_rs.best_estimator_}")
        print(f"hrchy_clustering_rs.best_params_: {hrchy_clustering_rs.best_params_}")
        print(f"hrchy_clustering_rs.best_score_: {hrchy_clustering_rs.best_score_}")
        print(f"hrchy_clustering_rs.best_estimator_.n_leaves_: {hrchy_clustering_rs.best_estimator_.n_leaves_}")
        print(f"hrchy_clustering_rs.best_estimator_.n_clusters_: {hrchy_clustering_rs.best_estimator_.n_clusters_}")
        print(f"np.unique(hrchy_clustering_rs.best_estimator_.labels_): {np.unique(hrchy_clustering_rs.best_estimator_.labels_, return_counts=True)}")
        print(f"hrchy_clustering_rs.best_estimator_.labels_: {hrchy_clustering_rs.best_estimator_.labels_}")
        print(f"hrchy_clustering_rs.n_features_in_: {hrchy_clustering_rs.n_features_in_}")
        print(f"hrchy_clustering_rs.feature_names_in_: {hrchy_clustering_rs.feature_names_in_}")
        print("-"*50)

    return hrchy_clustering_rs.best_estimator_


def hrchy_clustering_n_cluster_gs(X: "pd.DataFrame", data_mat_mode: str = "precomputed", verbose: int = 0):

    param_dict = {"n_clusters": range(2, 20), "affinity": [data_mat_mode],
                  "linkage": ["single", "complete", "average"]}
    cv = [(slice(None), slice(None))]
    hrchy_clustering_gs = GridSearchCV(estimator=AgglomerativeClustering(), param_grid=param_dict,
                                       scoring=calc_silhouette_label_freq_std, cv=cv, n_jobs=-1)
    hrchy_clustering_gs.fit(X)

    if verbose==1:
        print(f"hrchy_clustering_gs.best_estimator_: {hrchy_clustering_gs.best_estimator_}")
        print(f"hrchy_clustering_gs.best_params_: {hrchy_clustering_gs.best_params_}")
        print(f"hrchy_clustering_gs.best_score_: {hrchy_clustering_gs.best_score_}")
        print(f"hrchy_clustering_gs.best_estimator_.n_leaves_: {hrchy_clustering_gs.best_estimator_.n_leaves_}")
        print(f"hrchy_clustering_gs.best_estimator_.n_clusters_: {hrchy_clustering_gs.best_estimator_.n_clusters_}")
        print(f"np.unique(hrchy_clustering_gs.best_estimator_.labels_): {np.unique(hrchy_clustering_gs.best_estimator_.labels_, return_counts=True)}")
        print(f"hrchy_clustering_gs.best_estimator_.labels_: {hrchy_clustering_gs.best_estimator_.labels_}")
        print(f"hrchy_clustering_gs.n_features_in_: {hrchy_clustering_gs.n_features_in_}")
        print(f"hrchy_clustering_gs.feature_names_in_: {hrchy_clustering_gs.feature_names_in_}")
        print("-"*50)

    return hrchy_clustering_gs.best_estimator_


def obs_hrchy_cluster_instances(X: "pd.DataFrame", data_mat_mode: str = "precomputed", verbose: int = 1):

    for n in range(2, 20):
        hrchy_cluster = AgglomerativeClustering(n_clusters=n, linkage="complete", affinity=data_mat_mode, compute_distances=True)
        hrchy_cluster.fit(X)

        if verbose==1:
            print(f"hrchy_cluste.n_clusters_: {hrchy_cluster.n_clusters_}")
            print(f"hrchy_cluste.labels and whose number of instances: {np.unique(hrchy_cluster.labels_, return_counts=True)}")
            # print(f"(ticker, cluster label): {list(zip(X.index, hrchy_cluster.labels_))}")
            # print(f"The estimated number of connected components:{hrchy_cluster.n_connected_components_}")
            # print(f"hrchy_cluster.n_leaves_: {hrchy_cluster.n_leaves_}")
            # print(f"hrchy_cluster.n_features_in_: {hrchy_cluster.n_features_in_}")
            print("-"*50)


def hrchy_cluster_fixed_n_cluster(X: "pd.DataFrame", n: int, data_mat_mode: str = "precomputed", verbose: int = 1):

    hrchy_cluster = AgglomerativeClustering(n_clusters=n, linkage="complete", affinity=data_mat_mode)
    hrchy_cluster.fit(X)

    if verbose==1:
        print(f"hrchy_cluste.n_clusters_: {hrchy_cluster.n_clusters_}")
        print(f"hrchy_cluste.labels and whose instances: {np.unique(hrchy_cluster.labels_, return_counts=True)}")
        print(f"hrchy_cluster.n_leaves_: {hrchy_cluster.n_leaves_}")
        print(f"hrchy_cluster.n_features_in_: {hrchy_cluster.n_features_in_}")
        print("-"*50)

    return hrchy_cluster


# obs_hrchy_cluster_instances(distance_mat)
fixed_n_cluster = 11  # Determin by observe result of obs_hrchy_cluster_instances()
fixed_n_cluster_hrchy_cluster = hrchy_cluster_fixed_n_cluster(distance_mat, n=fixed_n_cluster)
# distance_threshold_hrchy_cluster = hrchy_clustering_distance_threshold_rs(dissimilarity_mat, verbose=0)
# n_cluster_hrchy_cluster = hrchy_clustering_n_cluster_gs(distance_mat, verbose=0)

## plot cluster label distribution

In [ ]:
def plot_cluster_labels_distribution(trained_cluster: "sklearn.cluster.<cluster_model>", cluster_name: str):
    x_major_locator = MultipleLocator(1)
    ax = plt.gca()
    ax.xaxis.set_major_locator(x_major_locator)
    plt.bar(np.unique(trained_cluster.labels_, return_counts=True)[0], np.unique(trained_cluster.labels_, return_counts=True)[1])
    plt.grid()
    plt.ylabel("instances in cluster")
    plt.xlabel("cluster label")
    plt.title(f"{cluster_name}\n {fig_title}")
    plt.show()  # findout elbow point
    plt.close()
    logging.info(f"cluster of each point distribution: {np.unique(trained_cluster.labels_, return_counts=True)}")

plot_cluster_labels_distribution(fixed_n_cluster_hrchy_cluster, f"Hirerarchy clustering with {fixed_n_cluster} n_clusters")
# plot_cluster_labels_distribution(n_cluster_hrchy_cluster, "Hirerarchy clustering with n_clusters")
# plot_cluster_labels_distribution(distance_threshold_hrchy_cluster, "Hirerarchy clustering with distance threshold")
# print(hrchy_cluster_labels_df.loc[hrchy_cluster_labels_df["hrchy_cluster_label"]==4, "items"].values)

## output cluster results

In [ ]:
cluster_items_dir = Path(data_cfg["DIRS"]["PIPELINE_DATA_DIR"])/f"{output_file_name}-cluster"
cluster_items_dir.mkdir(parents=True, exist_ok=True)

if fixed_n_cluster:
    output_cluster = fixed_n_cluster_hrchy_cluster  # the value of output_cluster depend on performance which shows in plot cluster label distribution
    output_cluster_name = f"corr_mat_hrchy_{fixed_n_cluster}_cluster"

hrchy_cluster_labels_df = pd.DataFrame(output_cluster.labels_, index=distance_mat.index, columns=[f"{output_cluster_name}_label"]).reset_index()
hrchy_cluster_labels_df.to_csv(cluster_items_dir/f"{output_file_name}-{corr_ser_clac_method}-{output_cluster_name}.csv")
logging.info(f"{output_file_name}-{corr_ser_clac_method}-{output_cluster_name}.csv has been save to {cluster_items_dir}")

# plot correlation coffecient distribution of data

In [ ]:
datasets = {'train_data--comb(150,2)': train_corr_series_concat, 'all_data--comb(445,2)': all_corr_series_concat, 'other_data--comb(295,2)': other_corr_series_concat}
etl_types = ["boxplot", "histogram", "qqplot", "Emprical Cumulative Density"]
fig, axes = plt.subplots(figsize=(20, 20),nrows=len(etl_types), ncols=len(datasets), sharex=False, sharey=False, dpi=100)

for row, etl_type in enumerate(etl_types):
    for col,dataset_key in enumerate(datasets):
        # print(row, etl_type, col, dataset_key, datasets[dataset_key])
        s = axes[row, col]
        s.set_title(f"{dataset_key}: \n{etl_type}", fontsize=24)
        if etl_type=="boxplot":
            s.boxplot(datasets[dataset_key], showmeans=True)
        elif etl_type=="histogram":
            s.hist(datasets[dataset_key], bins=[b/10 for b in range(-13,14)])
        elif etl_type=="qqplot":
            percents = [0.001, 0.2, 0.5, 0.8, 0.999]
            #x,y = [norm.ppf(p) for p in percents], [np.quantile(train_corr_series_concat, p) for p in percents]
            x,y = [norm.ppf(p) for p in percents], [np.quantile(datasets[dataset_key], p) for p in percents]
            sm.qqplot(datasets[dataset_key], line='q', ax=s)
            s.scatter(x,y, c='m', marker='x', s=300)
        elif etl_type=="Emprical Cumulative Density":
            pd.Series(datasets[dataset_key]).value_counts().sort_index().cumsum().plot(ax=s)

# 分開, 避免子圖標籤互相重疊
plt.tight_layout()
plt.savefig("./results/dataset_exploration.png")
plt.show()
plt.close()

In [ ]:
df = pd.DataFrame([[dataset_key, datasets[dataset_key].std()] for dataset_key in datasets], 
                  columns=['Dataset', 'Standard deviation'])
ax = sns.barplot(x='Dataset', y='Standard deviation', data=df)
ax.set_title('std of correlation')
ax.set(ylim=[0.47, 0.475])
ax.bar_label(ax.containers[0])
plt.xticks(rotation=60)
plt.savefig("./results/dataset_exploration_2.png")
plt.show()
plt.close()

In [ ]:
sns.distplot(train_corr_series_concat)
# plt.hist(train_corr_series, bins=[b/10 for b in range(-13,14)])

In [ ]:
train_corr_series_df = gen_corr_series(None, "train_dataset.csv", from_file=True, concat_all=False)
all_corr_series_df = gen_corr_series(None, "445_dataset.csv", from_file=True, concat_all=False)
other_corr_series_df = gen_corr_series(None, "295_dataset.csv", from_file=True, concat_all=False)

In [ ]:
datasets = {'train_data--comb(150,2)': train_corr_series_df, 'all_data--comb(445,2)': all_corr_series_df, 'other_data--comb(295,2)': other_corr_series_df}
etl_types = ["boxplot", "histogram"]
static_types = ["mean", "std"]
fig, axes = plt.subplots(figsize=(30, 30),nrows=len(list(product(etl_types, static_types))), ncols=len(datasets), sharex=False, sharey=False, dpi=100)

for row, (etl_type, static_type) in enumerate(product(etl_types, static_types)):
    for col,dataset_key in enumerate(datasets):
        s = axes[row, col]
        s.set_title(f"{dataset_key}: \n{etl_type}_{static_type}", fontsize=24)
        if etl_type=="boxplot":
            s.boxplot(datasets[dataset_key].iloc[:, ::5].describe().loc[static_type,:], showmeans=True)
        elif etl_type=="histogram":
            s.hist(datasets[dataset_key].iloc[:, ::5].describe().loc[static_type,:], bins=[b/10 for b in range(-13,14)])

fig.suptitle(f"Each correlation_series static property _20220718", fontsize=24)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
# 分開, 避免子圖標籤互相重疊
# plt.tight_layout()
plt.savefig("./results/dataset_exploration_3.png")
plt.show()
plt.close()

In [ ]:
display(train_corr_series_df)
display(train_corr_series_df.iloc[:,::5])
display(train_corr_series_df.iloc[:,::5].describe())
display(train_corr_series_df.iloc[:,::5].describe().loc['std',:])